### Prep

In [1]:
# install

!pip install -q ftfy regex tqdm pandas
!pip install -q git+https://github.com/openai/CLIP.git

In [2]:
# check GPU

!nvidia-smi -L

GPU 0: NVIDIA GeForce RTX 3080 (UUID: GPU-fc83ed6e-2e20-2518-fccc-97b999d65d26)


In [3]:
# imports

import numpy as np
import pandas as pd
import string
import time
import gc
import re
import os
import torch
import clip
from PIL import Image

In [4]:
# #@title ground truth

# truth = pd.read_csv('/workspace/LOC_val_solution.csv')
# truth = truth.sort_values(by = 'ImageId')
# truth['synset_labels'] = truth['PredictionString'].map(lambda x: [y for y in x.split(' ') if (y != '' and y[0] == 'n')])
# truth = truth.reset_index(drop = True)

In [5]:
# #@title ground truth

truth = pd.read_csv('/workspace/LOC_val_solution.csv')
truth = truth.sort_values(by = 'ImageId')
truth['synset_labels'] = truth['PredictionString'].map(lambda x: list(set([y for y in x.split(' ') if (y != '' and y[0] == 'n')]))[0])
truth = truth.reset_index(drop = True)

In [6]:
#@title image files

IMG_PATH = '/workspace/ILSVRC2012_img_val/'
filenames = truth['ImageId'].map(lambda x: f'{IMG_PATH}{x}.JPEG').reset_index(drop = True)

In [7]:
len(filenames)

50000

In [8]:
#@title count total num of parameters

def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

In [9]:
#@title synset labels mapping

PATH = '/workspace/synset_words.txt'

with open(PATH) as f:
    lines = [line.rstrip() for line in f]

synset_labels = pd.Series(lines).map(lambda x: x.split(' ', 1))
synset_labels = pd.DataFrame(synset_labels, columns = ['mapping'])
synset_labels['synset'] = synset_labels['mapping'].map(lambda x: x[0])
synset_labels['words'] = synset_labels['mapping'].map(lambda x: x[1].lower().translate(str.maketrans('', '', re.sub('\-', '', string.punctuation))))
synset_labels

,mapping,synset,words
0,"[n01440764, tench, Tinca tinca]",n01440764,tench tinca tinca
1,"[n01443537, goldfish, Carassius auratus]",n01443537,goldfish carassius auratus
2,"[n01484850, great white shark, white shark, ma...",n01484850,great white shark white shark man-eater man-ea...
3,"[n01491361, tiger shark, Galeocerdo cuvieri]",n01491361,tiger shark galeocerdo cuvieri
4,"[n01494475, hammerhead, hammerhead shark]",n01494475,hammerhead hammerhead shark
...,...,...,...
995,"[n13044778, earthstar]",n13044778,earthstar
996,"[n13052670, hen-of-the-woods, hen of the woods...",n13052670,hen-of-the-woods hen of the woods polyporus fr...
997,"[n13054560, bolete]",n13054560,bolete
998,"[n13133613, ear, spike, capitulum]",n13133613,ear spike capitulum


In [10]:
#@title pretrained official CLIP models

clip.available_models()

['RN50',
 'RN101',
 'RN50x4',
 'RN50x16',
 'RN50x64',
 'ViT-B/32',
 'ViT-B/16',
 'ViT-L/14',
 'ViT-L/14@336px']

In [11]:
# #@title inferrance pipeline

# def infer_pipe(pretrained_model, exp, N = 1000, target = 'words', num_batch = 50, filenames = filenames):
    
#     if not os.path.exists(f'/workspace/preds_{exp}'):
#         os.mkdir(f'/workspace/preds_{exp}')

#     device = "cuda" if torch.cuda.is_available() else "cpu"
#     model, preprocess = clip.load(str(pretrained_model), device=device)
#     print(f'total number of parameters: {count_parameters(model)}.')

#     labels = synset_labels[target]

#     with torch.no_grad():
#       text = clip.tokenize(labels).to(device)
#       text_features = model.encode_text(text)

#       for i in range(num_batch):

#         preds = []

#         start = time.time()

#         for j in range(i * N, i * N + N):

#           PATH = filenames[j]
#           image = preprocess(Image.open(str(PATH))).unsqueeze(0).to(device)

#           image_features = model.encode_image(image)

#           logits_per_image, logits_per_text = model(image, text)
#           probs = logits_per_image.softmax(dim=-1).cpu().numpy()

#           preds.append(probs[0])

#         preds_batch = pd.DataFrame(preds)
#         preds_batch.to_csv(f'/workspace/preds_{exp}/preds_{i+1}k.csv', index=False)
#         duration = time.time() - start
#         print(f'preds_{i+1}k.csv saved!', duration)

#         image, image_features, preds_batch = None, None, None
#         gc.collect()

#@title inferrance pipeline

def infer_pipe(pretrained_model, exp, N = 1000, target = 'words', num_batch = 50, filenames = filenames):
    
    if not os.path.exists(f'/workspace/preds_{exp}'):
        os.mkdir(f'/workspace/preds_{exp}')

    device = "cuda" if torch.cuda.is_available() else "cpu"
    model, preprocess = clip.load(str(pretrained_model), device=device)
    print(f'total number of parameters: {count_parameters(model)}.')

    labels = synset_labels[target]

    with torch.no_grad():
      text = clip.tokenize(labels).to(device)
      text_features = model.encode_text(text)

      for i in range(num_batch):
        
        preds = []
        
        start = time.time()

        for j in range(i * N, i * N + N):
        # for j in filenames.index:

          PATH = filenames[j]
          image = preprocess(Image.open(str(PATH))).unsqueeze(0).to(device)

          image_features = model.encode_image(image)

          logits_per_image, logits_per_text = model(image, text)
          probs = logits_per_image.softmax(dim=-1).cpu().numpy()

          preds.append(probs[0])

        preds_batch = pd.DataFrame(preds)
        preds_batch.to_csv(f'/workspace/preds_{exp}/preds_{i+1}k.csv', index=False)
        duration = time.time() - start
        print(f'preds_{i+1}k.csv saved!', duration)

        image, image_features, preds_batch = None, None, None
        gc.collect()

In [1]:
# #@title other utility

# import shutil

# shutil.rmtree('/workspace/preds_09')

### Experiment 1

In [9]:
#@title set device & create model

device = "cuda" if torch.cuda.is_available() else "cpu"
model, preprocess = clip.load("ViT-B/32", device=device)

100%|███████████████████████████████████████| 338M/338M [00:15<00:00, 22.6MiB/s]


In [23]:
#@title CLIP classifier examples

N = 1000

labels = synset_labels['words']
text = clip.tokenize(labels).to(device)

with torch.no_grad():
  text_features = model.encode_text(text)
  
  for i in range(50):
    
    preds = []

    start = time.time()
    
    for j in range(i * N, i * N + N):

      PATH = filenames[j]
      image = preprocess(Image.open(str(PATH))).unsqueeze(0).to(device)

      image_features = model.encode_image(image)
      
      logits_per_image, logits_per_text = model(image, text)
      probs = logits_per_image.softmax(dim=-1).cpu().numpy()

      preds.append(probs[0])
    
    preds_batch = pd.DataFrame(preds)
    preds_batch.to_csv(f'/workspace/preds_01/preds_{i+1}k.csv', index=False)
    duration = time.time() - start
    print(f'preds_{i+1}k.csv saved!', duration)
    
    image, image_features, preds_batch = None, None, None
    gc.collect()

preds_1k.csv saved! 2.511321544647217
preds_2k.csv saved! 2.470968246459961


### Experiment 2

In [6]:
#@title synset labels mapping

PATH = '/workspace/synset_words.txt'

with open(PATH) as f:
    lines = [line.rstrip() for line in f]

synset_labels = pd.Series(lines).map(lambda x: x.split(' ', 1))
synset_labels = pd.DataFrame(synset_labels, columns = ['mapping'])
synset_labels['synset'] = synset_labels['mapping'].map(lambda x: x[0])

# exp. 2, taking only the first set of description as embedding target

synset_labels['words'] = synset_labels['mapping'].map(lambda x: x[1].lower().translate(str.maketrans('', '', re.sub('\-|\,', '', string.punctuation))))
synset_labels['single_term'] = synset_labels['words'].map(lambda x: x.split(',')[0])
synset_labels

,mapping,synset,words,single_term
0,"[n01440764, tench, Tinca tinca]",n01440764,"tench, tinca tinca",tench
1,"[n01443537, goldfish, Carassius auratus]",n01443537,"goldfish, carassius auratus",goldfish
2,"[n01484850, great white shark, white shark, ma...",n01484850,"great white shark, white shark, man-eater, man...",great white shark
3,"[n01491361, tiger shark, Galeocerdo cuvieri]",n01491361,"tiger shark, galeocerdo cuvieri",tiger shark
4,"[n01494475, hammerhead, hammerhead shark]",n01494475,"hammerhead, hammerhead shark",hammerhead
...,...,...,...,...
995,"[n13044778, earthstar]",n13044778,earthstar,earthstar
996,"[n13052670, hen-of-the-woods, hen of the woods...",n13052670,"hen-of-the-woods, hen of the woods, polyporus ...",hen-of-the-woods
997,"[n13054560, bolete]",n13054560,bolete,bolete
998,"[n13133613, ear, spike, capitulum]",n13133613,"ear, spike, capitulum",ear


In [30]:
#@title image files

IMG_PATH = '/workspace/ILSVRC2012_img_val/'
filenames = truth['ImageId'].map(lambda x: f'{IMG_PATH}{x}.JPEG').reset_index(drop = True)

In [8]:
#@title set device & create model

device = "cuda" if torch.cuda.is_available() else "cpu"
model, preprocess = clip.load("ViT-B/32", device=device)

/opt/conda/lib/python3.7/site-packages/clip/clip.py:57: UserWarning: /root/.cache/clip/ViT-B-32.pt exists, but the SHA256 checksum does not match; re-downloading the file
  warnings.warn(f"{download_target} exists, but the SHA256 checksum does not match; re-downloading the file")
100%|███████████████████████████████████████| 338M/338M [00:34<00:00, 10.2MiB/s]


In [11]:
#@title CLIP classifier examples

N = 1000

labels = synset_labels['single_term']
text = clip.tokenize(labels).to(device)

with torch.no_grad():
  text_features = model.encode_text(text)
  
  for i in range(5):
    
    preds = []

    start = time.time()
    
    for j in range(i * N, i * N + N):

      PATH = filenames[j]
      image = preprocess(Image.open(str(PATH))).unsqueeze(0).to(device)

      image_features = model.encode_image(image)
      
      logits_per_image, logits_per_text = model(image, text)
      probs = logits_per_image.softmax(dim=-1).cpu().numpy()

      preds.append(probs[0])
    
    preds_batch = pd.DataFrame(preds)
    preds_batch.to_csv(f'/workspace/preds_02/preds_{i+1}k.csv', index=False)
    duration = time.time() - start
    print(f'preds_{i+1}k.csv saved!', duration)
    
    image, image_features, preds_batch = None, None, None
    gc.collect()

preds_1k.csv saved! 255.75778365135193
preds_2k.csv saved! 258.0606269836426
preds_3k.csv saved! 258.1417591571808
preds_4k.csv saved! 257.7300350666046
preds_5k.csv saved! 257.8815743923187


### Experiment 3

In [30]:
infer_pipe(pretrained_model = 'RN50x64', exp = '03')

100%|█████████████████████████████████████| 1.26G/1.26G [02:13<00:00, 10.2MiB/s]


total number of parameters: 623258305.
preds_1k.csv saved! 723.5701186656952
preds_2k.csv saved! 722.6463541984558
preds_3k.csv saved! 722.7152602672577
preds_4k.csv saved! 722.4449908733368
preds_5k.csv saved! 722.5220715999603
preds_6k.csv saved! 722.4453015327454
preds_7k.csv saved! 722.4998171329498
preds_8k.csv saved! 722.4210135936737
preds_9k.csv saved! 722.6207797527313
preds_10k.csv saved! 722.7291519641876
preds_11k.csv saved! 722.7533469200134
preds_12k.csv saved! 722.6446599960327
preds_13k.csv saved! 723.0684010982513
preds_14k.csv saved! 722.9709951877594
preds_15k.csv saved! 722.9316759109497
preds_16k.csv saved! 722.7477905750275
preds_17k.csv saved! 722.911239862442
preds_18k.csv saved! 723.1533415317535
preds_19k.csv saved! 722.9051961898804
preds_20k.csv saved! 722.8734028339386
preds_21k.csv saved! 722.6805186271667
preds_22k.csv saved! 722.6690745353699
preds_23k.csv saved! 722.8009505271912
preds_24k.csv saved! 722.669401884079
preds_25k.csv saved! 722.73649621009

### Experiment 4

In [34]:
infer_pipe(pretrained_model = 'ViT-L/14@336px', exp = '04')

100%|███████████████████████████████████████| 891M/891M [02:22<00:00, 6.53MiB/s]


total number of parameters: 427944193.
preds_1k.csv saved! 475.2574827671051
preds_2k.csv saved! 475.2695982456207
preds_3k.csv saved! 475.2157151699066
preds_4k.csv saved! 474.95254254341125
preds_5k.csv saved! 475.0481209754944
preds_6k.csv saved! 475.0319013595581
preds_10k.csv saved! 475.2324342727661
preds_11k.csv saved! 475.253778219223
preds_12k.csv saved! 475.0516633987427
preds_13k.csv saved! 475.56936979293823
preds_14k.csv saved! 475.35669016838074
preds_15k.csv saved! 475.4551296234131
preds_16k.csv saved! 475.12925815582275
preds_17k.csv saved! 475.2387068271637
preds_18k.csv saved! 475.6053817272186
preds_19k.csv saved! 475.27045607566833
preds_20k.csv saved! 475.3283622264862
preds_21k.csv saved! 475.178674697876
preds_22k.csv saved! 475.10363578796387
preds_23k.csv saved! 475.28603053092957
preds_24k.csv saved! 475.2000398635864
preds_25k.csv saved! 475.2415509223938
preds_26k.csv saved! 475.27772307395935
preds_27k.csv saved! 475.19736766815186
preds_28k.csv saved! 475

### Experiment 5

In [14]:
synset_labels.loc[168, 'words'] = 'redbone coonhound'

In [15]:
synset_labels[synset_labels['words'] == 'redbone coonhound']

,mapping,synset,words
168,"[n02090379, redbone]",n02090379,redbone coonhound


In [16]:
len(synset_labels)

1000

In [15]:
truth[truth['synset_labels'] == 'n02090379'].index

Int64Index([ 1266,  1362,  1861,  2932,  5492,  5807,  6395,  8165,  8271,
             8697,  9347, 11112, 11175, 11797, 15884, 17002, 18326, 20738,
            21574, 23100, 23154, 24042, 24485, 24949, 25695, 26735, 28005,
            30695, 31820, 31948, 32745, 33048, 33093, 34023, 35487, 35688,
            35911, 37854, 38736, 39021, 40355, 41590, 41687, 42441, 43387,
            43887, 44443, 46431, 46752, 49765],
           dtype='int64')

In [16]:
filenames

0        /workspace/ILSVRC2012_img_val/ILSVRC2012_val_0...
1        /workspace/ILSVRC2012_img_val/ILSVRC2012_val_0...
2        /workspace/ILSVRC2012_img_val/ILSVRC2012_val_0...
3        /workspace/ILSVRC2012_img_val/ILSVRC2012_val_0...
4        /workspace/ILSVRC2012_img_val/ILSVRC2012_val_0...
                               ...                        
49995    /workspace/ILSVRC2012_img_val/ILSVRC2012_val_0...
49996    /workspace/ILSVRC2012_img_val/ILSVRC2012_val_0...
49997    /workspace/ILSVRC2012_img_val/ILSVRC2012_val_0...
49998    /workspace/ILSVRC2012_img_val/ILSVRC2012_val_0...
49999    /workspace/ILSVRC2012_img_val/ILSVRC2012_val_0...
Name: ImageId, Length: 50000, dtype: object

In [17]:
filenames_n02090379 = filenames[filenames.index.isin(truth[truth['synset_labels'] == 'n02090379'].index)]

In [21]:
infer_pipe(pretrained_model = 'ViT-B/32', N = 50, target = 'words', num_batch = 1, exp = '05', filenames = filenames_n02090379)

total number of parameters: 151277313.
preds_1k.csv saved! 12.663532972335815


In [24]:
infer_pipe(pretrained_model = 'ViT-B/32', exp = '05A', filenames = filenames)

total number of parameters: 151277313.
preds_1k.csv saved! 265.68636870384216
preds_2k.csv saved! 266.8919777870178
preds_3k.csv saved! 266.77229833602905
preds_4k.csv saved! 266.4753510951996
preds_5k.csv saved! 266.63124656677246
preds_6k.csv saved! 266.5339879989624
preds_7k.csv saved! 266.6189911365509
preds_8k.csv saved! 266.4862344264984
preds_9k.csv saved! 266.6978032588959
preds_10k.csv saved! 266.9317035675049
preds_11k.csv saved! 266.71081709861755
preds_12k.csv saved! 266.58709383010864
preds_13k.csv saved! 267.00905084609985
preds_14k.csv saved! 266.9247872829437
preds_15k.csv saved! 266.792197227478
preds_16k.csv saved! 266.63494181632996
preds_17k.csv saved! 266.715003490448
preds_18k.csv saved! 267.1524381637573
preds_19k.csv saved! 266.9397072792053
preds_20k.csv saved! 266.82797050476074
preds_21k.csv saved! 266.70872497558594
preds_22k.csv saved! 266.56342482566833
preds_23k.csv saved! 266.7542688846588
preds_24k.csv saved! 266.6470172405243
preds_25k.csv saved! 266.6

### Experiment 6

In [22]:
infer_pipe(pretrained_model = 'ViT-L/14@336px', N = 50, target = 'words', num_batch = 1, exp = '06', filenames = filenames_n02090379)

100%|███████████████████████████████████████| 891M/891M [01:33<00:00, 10.0MiB/s]


total number of parameters: 427944193.
preds_1k.csv saved! 23.373618125915527


In [ ]:
infer_pipe(pretrained_model = 'ViT-L/14@336px', exp = '06A', filenames = filenames)

total number of parameters: 427944193.
preds_1k.csv saved! 476.6822860240936
preds_2k.csv saved! 475.59051537513733
preds_3k.csv saved! 475.5950286388397
preds_4k.csv saved! 475.1951127052307
preds_5k.csv saved! 475.22230219841003
preds_6k.csv saved! 475.3377058506012
preds_7k.csv saved! 475.3007035255432
preds_8k.csv saved! 475.131290435791
preds_9k.csv saved! 475.33287811279297
preds_10k.csv saved! 475.40201592445374
preds_11k.csv saved! 475.4406340122223
preds_12k.csv saved! 475.21579790115356
preds_13k.csv saved! 475.7755637168884
preds_14k.csv saved! 475.6451003551483
preds_15k.csv saved! 475.67120838165283
preds_16k.csv saved! 475.418172121048
preds_17k.csv saved! 475.6698079109192
preds_18k.csv saved! 475.9453375339508
preds_19k.csv saved! 475.7512104511261
preds_20k.csv saved! 475.70912170410156
preds_21k.csv saved! 475.46031641960144
preds_30k.csv saved! 475.59962344169617
preds_31k.csv saved! 475.6789562702179
preds_32k.csv saved! 475.38918471336365
preds_33k.csv saved! 475.5

### Experiment 7

In [12]:
idx = synset_labels[synset_labels['words'] == 'redbone'].index[0]
synset_labels.loc[idx, 'words'] = 'redbone coonhound'
synset_labels.iloc[idx]

mapping    [n02090379, redbone]
synset                n02090379
words         redbone coonhound
Name: 168, dtype: object

In [13]:
idx = synset_labels[synset_labels['words'] == 'stole'].index[0]
synset_labels.loc[idx, 'words'] = 'stole scarf shawl'
synset_labels.iloc[idx]

mapping    [n04325704, stole]
synset              n04325704
words       stole scarf shawl
Name: 824, dtype: object

In [14]:
infer_pipe(pretrained_model = 'ViT-L/14@336px', exp = '07', filenames = filenames)

total number of parameters: 427944193.
preds_1k.csv saved! 476.6612913608551
preds_2k.csv saved! 475.4137833118439
preds_3k.csv saved! 475.5848217010498
preds_4k.csv saved! 475.2093925476074
preds_5k.csv saved! 475.3602192401886
preds_6k.csv saved! 475.25270771980286
preds_7k.csv saved! 475.3604357242584
preds_8k.csv saved! 475.32327938079834
preds_9k.csv saved! 475.96876096725464
preds_10k.csv saved! 476.3752210140228
preds_11k.csv saved! 483.757600069046
preds_12k.csv saved! 483.70049500465393
preds_13k.csv saved! 484.03154587745667
preds_14k.csv saved! 483.89209055900574
preds_15k.csv saved! 483.99609088897705
preds_16k.csv saved! 483.69059920310974
preds_17k.csv saved! 483.87818217277527
preds_18k.csv saved! 484.1745665073395
preds_19k.csv saved! 484.0274488925934
preds_20k.csv saved! 483.9296669960022
preds_21k.csv saved! 483.8079788684845
preds_22k.csv saved! 483.6524531841278
preds_23k.csv saved! 483.7693841457367
preds_24k.csv saved! 483.112149477005
preds_25k.csv saved! 483.09

### Experiment 8

In [ ]:
idx = synset_labels[synset_labels['words'] == 'redbone'].index[0]
synset_labels.loc[idx, 'words'] = 'redbone coonhound'
synset_labels.iloc[idx]

In [ ]:
idx = synset_labels[synset_labels['words'] == 'stole'].index[0]
synset_labels.loc[idx, 'words'] = 'stole scarf shawl'
synset_labels.iloc[idx]

In [16]:
idx = synset_labels[synset_labels['words'] == 'yawl'].index[0]
synset_labels.loc[idx, 'words'] = 'yawl a two-masted fore-and-aft-rigged sailboat with the mizzenmast stepped far aft so that the mizzen boom overhangs the stern'
synset_labels.iloc[idx]

mapping                                    [n04612504, yawl]
synset                                             n04612504
words      yawl a two-masted fore-and-aft-rigged sailboat...
Name: 914, dtype: object

In [18]:
infer_pipe(pretrained_model = 'ViT-L/14@336px', exp = '08', filenames = filenames)

total number of parameters: 427944193.
preds_1k.csv saved! 474.7779424190521
preds_2k.csv saved! 475.6876986026764
preds_3k.csv saved! 475.6837935447693
preds_4k.csv saved! 475.3853225708008
preds_6k.csv saved! 475.690083026886
preds_7k.csv saved! 475.9188516139984
preds_8k.csv saved! 475.7276632785797
preds_9k.csv saved! 475.50074100494385
preds_10k.csv saved! 475.622759103775
preds_11k.csv saved! 475.6400909423828
preds_12k.csv saved! 475.5251898765564
preds_13k.csv saved! 476.02331614494324
preds_14k.csv saved! 475.9444818496704
preds_15k.csv saved! 475.89088201522827
preds_16k.csv saved! 475.61802315711975
preds_17k.csv saved! 475.66472816467285
preds_18k.csv saved! 476.00608134269714
preds_19k.csv saved! 475.75237822532654
preds_21k.csv saved! 475.5383687019348
preds_22k.csv saved! 475.4537236690521
preds_23k.csv saved! 475.6552028656006
preds_24k.csv saved! 475.583535194397
preds_25k.csv saved! 475.62280106544495
preds_26k.csv saved! 475.66553688049316
preds_27k.csv saved! 475.41

### Experiment 9

In [12]:
idx = synset_labels[synset_labels['words'] == 'redbone'].index[0]
synset_labels.loc[idx, 'words'] = 'redbone coonhound'
synset_labels.iloc[idx]

mapping    [n02090379, redbone]
synset                n02090379
words         redbone coonhound
Name: 168, dtype: object

In [13]:
idx = synset_labels[synset_labels['words'] == 'stole'].index[0]
synset_labels.loc[idx, 'words'] = 'stole scarf shawl'
synset_labels.iloc[idx]

mapping    [n04325704, stole]
synset              n04325704
words       stole scarf shawl
Name: 824, dtype: object

In [14]:
idx = synset_labels[synset_labels['words'] == 'yawl'].index[0]
synset_labels.loc[idx, 'words'] = 'yawl a two-masted fore-and-aft-rigged sailboat with the mizzenmast stepped far aft so that the mizzen boom overhangs the stern'
synset_labels.iloc[idx]

mapping                                    [n04612504, yawl]
synset                                             n04612504
words      yawl a two-masted fore-and-aft-rigged sailboat...
Name: 914, dtype: object

In [15]:
idx = synset_labels[synset_labels['words'] == 'schooner'].index[0]
synset_labels.loc[idx, 'words'] = 'schooner a sailing ship with two or more masts typically with the foremast smaller than the mainmast and having gaff-rigged lower masts'
synset_labels.iloc[idx]

mapping                                [n04147183, schooner]
synset                                             n04147183
words      schooner a sailing ship with two or more masts...
Name: 780, dtype: object

In [16]:
infer_pipe(pretrained_model = 'ViT-L/14@336px', exp = '09', filenames = filenames)

100%|███████████████████████████████████████| 891M/891M [02:17<00:00, 6.81MiB/s]


total number of parameters: 427944193.
preds_1k.csv saved! 484.90875601768494
preds_2k.csv saved! 485.01882815361023
preds_3k.csv saved! 483.9597463607788
preds_4k.csv saved! 483.7469074726105
preds_5k.csv saved! 483.85817670822144
preds_6k.csv saved! 483.8070938587189
preds_10k.csv saved! 484.00054025650024
preds_11k.csv saved! 484.0163688659668
preds_12k.csv saved! 483.86414909362793
preds_13k.csv saved! 484.1830222606659
preds_14k.csv saved! 484.13636326789856
preds_15k.csv saved! 484.1592810153961
preds_16k.csv saved! 483.8264248371124
preds_17k.csv saved! 484.0268671512604
preds_18k.csv saved! 484.3898878097534
preds_19k.csv saved! 484.14547777175903
preds_20k.csv saved! 483.83469581604004
preds_21k.csv saved! 482.966828584671
preds_22k.csv saved! 482.9562542438507
preds_23k.csv saved! 483.3921241760254
preds_24k.csv saved! 483.8566720485687
preds_25k.csv saved! 483.9471228122711
preds_26k.csv saved! 483.9211070537567
preds_27k.csv saved! 483.85203313827515
preds_28k.csv saved! 48

### Experiment 10

In [ ]:
idx = synset_labels[synset_labels['words'] == 'redbone'].index[0]
synset_labels.loc[idx, 'words'] = 'redbone coonhound'
synset_labels.iloc[idx]

In [ ]:
idx = synset_labels[synset_labels['words'] == 'stole'].index[0]
synset_labels.loc[idx, 'words'] = 'scarf shawl'
synset_labels.iloc[idx]

In [ ]:
idx = synset_labels[synset_labels['words'] == 'yawl'].index[0]
synset_labels.loc[idx, 'words'] = 'yawl two masted fore aft rigged sailboat mizzen boom overhangs stern'
synset_labels.iloc[idx]

In [ ]:
idx = synset_labels[synset_labels['words'] == 'schooner'].index[0]
synset_labels.loc[idx, 'words'] = 'schooner sailing ship the foremast smaller than the mainmast gaff rigged lower masts'
synset_labels.iloc[idx]

In [ ]:
idx = synset_labels[synset_labels['words'] == 'nipple'].index[0]
synset_labels.loc[idx, 'words'] = 'baby feeding bottle'
synset_labels.iloc[idx]

In [ ]:
infer_pipe(pretrained_model = 'ViT-L/14@336px', exp = '10', filenames = filenames)